In [1]:
%load_ext autoreload
%autoreload 3

In [2]:
import numpy as np
from minitorch.tensor.tensor import Tensor
from minitorch.nn.layers import (Linear,
                                Dropout,
                                Sequential,
                                Residual,
                                BatchNormalization,
                                LayerNormalization)

In [3]:
batch, in_features, out_features = 5,6,5
layer1 = Linear(in_features, out_features, bias=True)
layer2 = Linear(out_features, in_features, bias=True)
layer3 = Linear(in_features, 4, bias=True)

drop_out = Dropout(0.2, training=True)

d_model = Sequential([layer1,drop_out, layer2,drop_out, layer3])
nd_model = Sequential([layer1, layer2, layer3])

x = Tensor(np.random.randn(batch, in_features))
print(x)
d_out = d_model(x)
nd_out = nd_model(x)

Tensor(data=[[ 1.78155558e+00 -2.06984730e+00  7.47858157e-01 -2.60976894e-01
  -1.38405754e+00  1.31465025e+00]
 [-1.47097883e-01  2.14242717e+00 -1.70399194e+00  2.04098982e-01
  -7.96782925e-01  1.11450121e+00]
 [ 2.04086384e-03  8.88168716e-02 -4.28728900e-01  5.79349621e-01
   5.02843628e-01  5.46330184e-01]
 [-7.72783475e-01 -1.99241975e-02  1.15874036e+00 -1.73275252e-01
   3.68702551e-01  7.98850582e-01]
 [ 4.02954116e-01 -9.31626478e-01 -1.14706620e+00 -5.11614065e-01
   3.79190435e-01  1.38519804e-01]])


In [4]:
weight, bias = layer1.parameters()

In [5]:
layer1.__repr__()

'Linear(in_features=6, out_features=5, bias=True)'

In [6]:
bn = BatchNormalization(num_features=d_out.shape[-1])
out_norm = bn(d_out)

In [7]:
lm = LayerNormalization(num_features=d_out.shape[-1])
out_lnorm = lm(d_out)

In [8]:
out_norm, out_lnorm

(Tensor(data=[[ 1.12492785  0.54550954 -1.20440288  1.00493992]
  [-1.60684799  0.69069154  1.58056824 -0.84682306]
  [-0.01965291 -1.68024021 -0.13631263 -1.23028523]
  [ 0.36564721 -0.16662922 -0.16299679  0.2331118 ]
  [ 0.13592584  0.61066835 -0.07685594  0.83905658]], shape=(5, 4), grad_info= requires_grad=True),
 Tensor(data=[[ 1.43410922 -0.33948102 -0.88565259 -0.2089756 ]
  [-1.41740584  0.3394242   0.92286251  0.15511913]
  [ 0.73052329 -0.99377602  0.98110735 -0.71785461]
  [ 1.48172841 -0.7114378  -0.35928526 -0.41100535]
  [ 1.35740922 -0.87382482 -0.61094813  0.12736374]], shape=(5, 4), grad_info= requires_grad=True))

In [9]:
TOLERANCE = 1e-3

def unit_test():
    print("="*50)
    print("🧪Running unit tests for Linear layer...")
    in_features, out_features = 784, 256
    layer = Linear(in_features, out_features, bias=True)
    
    #* test layer creation
    assert layer.weight.shape == (out_features, in_features), "Weight shape is incorrect."
    assert layer.bias.shape == (out_features,), "Bias shape is incorrect."
    assert layer.weight.requires_grad, "Weight should require gradients."
    assert layer.bias.requires_grad, "Bias should require gradients."
    print("🧪 Layer creation tests passed ...")
    
    #* test Xavier initialization
    weight_std = np.std(layer.weight.data)
    expected_std = (1.0 / in_features) ** 0.5
    assert np.isclose(weight_std, expected_std, atol=TOLERANCE), "Weight initialization is not Xavier."
    print("🧪 Xavier initialization tests passed ...")
    
    #* bias tests
    assert np.all(layer.bias.data == 0), "Bias should be initialized to zeros."
    layer_no_bias = Linear(in_features, out_features, bias=False)
    assert layer_no_bias.bias is None, "Bias should be None when bias=False."
    params = layer_no_bias.parameters()
    assert len(params) == 1 and params[0] is layer_no_bias.weight, "Parameters should only include weight when bias=False."
    print("🧪 Bias initialization tests passed ...")
    
    #* forward pass tests
    batch_size = 10
    x = Tensor(np.random.randn(batch_size, in_features))
    output = layer(x)
    assert output.shape == (batch_size, out_features), f"expected output shape {(batch_size, out_features)}, got {output.shape}."
    print("🧪 Forward pass tests passed ...")
    print("✅ All unit tests passed!")
    print("="*50)
    
unit_test()

🧪Running unit tests for Linear layer...
🧪 Layer creation tests passed ...
🧪 Xavier initialization tests passed ...
🧪 Bias initialization tests passed ...
🧪 Forward pass tests passed ...
✅ All unit tests passed!


In [10]:
def test_edge_cases_linear():
    """🔬 Test Linear layer edge cases."""
    print("🔬 Edge Case Tests: Linear Layer...")

    layer = Linear(10, 5)

    # Test single sample (should handle 2D input)
    x_2d = Tensor(np.random.randn(1, 10))
    y = layer.forward(x_2d)
    assert y.shape == (1, 5), "Should handle single sample"
    print("🧪 Single sample handled correctly...")
    
    # Test zero batch size (edge case)
    x_empty = Tensor(np.random.randn(0, 10))
    y_empty = layer.forward(x_empty)
    assert y_empty.shape == (0, 5), "Should handle empty batch"
    print("🧪 Empty batch handled correctly...")
    
    # Test numerical stability with large weights
    layer_large = Linear(10, 5)
    layer_large.weight.data = np.ones((5,10)) * 100  # Large but not extreme
    x = Tensor(np.ones((1, 10)))
    y = layer_large.forward(x)
    assert not np.any(np.isnan(y.data)), "Should not produce NaN with large weights"
    assert not np.any(np.isinf(y.data)), "Should not produce Inf with large weights"
    print("🧪 Numerical stability with large weights handled correctly...")
    
    # Test with no bias
    layer_no_bias = Linear(10, 5, bias=False)
    x = Tensor(np.random.randn(4, 10))
    y = layer_no_bias.forward(x)
    assert y.shape == (4, 5), "Should work without bias"
    print("🧪 No bias case handled correctly...")
    print("✅ Edge cases handled correctly!")
    print("="*50)
    
test_edge_cases_linear()

🔬 Edge Case Tests: Linear Layer...
🧪 Single sample handled correctly...
🧪 Empty batch handled correctly...
🧪 Numerical stability with large weights handled correctly...
🧪 No bias case handled correctly...
✅ Edge cases handled correctly!


In [11]:
def test_unit_dropout():
    """🧪 Unit tests for Dropout layer."""
    print("="*50)
    print("🧪Running unit tests for Dropout layer...")
    
    #* test layer creation
    p = 0.3
    layer = Dropout(p)
    assert layer.p == p, "Dropout probability not set correctly."
    print("🧪 Layer creation tests passed ...")
    
    #* test forward pass in training mode
    batch_size, features = 10, 5
    x = Tensor(np.ones((batch_size, features)))
    output = layer(x)
    dropped_elements = np.sum(output.data == 0)
    expected_dropped = int(batch_size * features * p)
    assert abs(dropped_elements - expected_dropped) < batch_size * features * 0.1, "Dropout did not drop the expected number of elements."
    print("🧪 Forward pass in training mode tests passed ...")
    
    #* test forward pass in evaluation mode
    output_eval = layer(x, training=False)
    assert np.all(output_eval.data == x.data), "Dropout should not modify input in evaluation mode."
    print("🧪 Forward pass in evaluation mode tests passed ...")
    
    #* test training mode with p=0 (no dropout)
    layer_no_dropout = Dropout(0.0)
    output_no_dropout = layer_no_dropout(x)
    assert np.all(output_no_dropout.data == x.data), "Dropout with p=0 should not modify input."
    print("🧪 No dropout (p=0) tests passed ...")
    
    #* test training mode with p=1 (all dropped)
    layer_all_dropout = Dropout(1.0 - 1e-6) # avoid exact 1.0 to prevent division by zero
    output_all_dropout = layer_all_dropout(x)
    assert np.all(output_all_dropout.data == 0), "Dropout with p=1 should drop all elements."
    print("🧪 All dropout (p=1) tests passed ...")
    
    #* test training mode with partial dropout
    #* this probabilistic test may fail occasionally due to randomness
    #* so we test statistical properties instead of exact values
    np.random.seed(42) # for reproducibility
    x_large = Tensor(np.ones((1000,))) # large input for statistical test
    layer_partial_dropout = Dropout(0.5)
    y_train = layer_partial_dropout(x_large, training=True)
    
    #* count non-dropped elements(approx 50% should remain)
    non_zero_count = np.count_nonzero(y_train.data)
    expected = 500
    #* Use 3-sigma bounds: std = sqrt(n*p*(1-p)) = sqrt(1000*0.5*0.5) ≈ 15.8
    std_error = np.sqrt(1000 * 0.5 * 0.5)
    lower_bound = expected - 3 * std_error  # ≈ 453
    upper_bound = expected + 3 * std_error  # ≈ 547
    assert lower_bound < non_zero_count < upper_bound, \
        f"Expected {expected}±{3*std_error:.0f} survivors, got {non_zero_count}"
    print("🧪 Partial dropout tests passed ...")
    
    # Test scaling (surviving elements should be scaled by 1/(1-p) = 2.0)
    surviving_values = y_train.data[y_train.data != 0]
    expected_value = 2.0  # 1.0 / (1 - 0.5)
    assert np.allclose(surviving_values, expected_value), f"Surviving values should be {expected_value}"
    print("🧪 Dropout scaling tests passed ...")
    
    print("🧪 Dropout layer tests passed ...")
    print("✅ All unit tests passed!")
    print("="*50)
    
test_unit_dropout()

🧪Running unit tests for Dropout layer...
🧪 Layer creation tests passed ...


AssertionError: Dropout did not drop the expected number of elements.

In [14]:
import numpy as np
import pytest


# ----------------------
# Linear Layer Tests
# ----------------------

def test_linear_forward_shape():
    layer = Linear(4, 3)
    x = Tensor(np.random.randn(5, 4))

    out = layer(x)

    assert out.shape == (5, 3)


def test_linear_backward_pass():
    layer = Linear(4, 2, bias=True)
    x = Tensor(np.random.randn(6, 4), requires_grad=True)

    out = layer(x)
    loss = out.sum()
    loss.backward()

    assert x.grad is not None
    assert layer.weight.grad is not None
    assert layer.bias.grad is not None


# ----------------------
# Dropout Tests
# ----------------------

def test_dropout_training_changes_values():
    dropout = Dropout(p=0.5)
    dropout.train()

    x = Tensor(np.ones((100, 10)))
    out = dropout(x)

    # Expect some zeros during training
    assert (out.data == 0).any()


def test_dropout_eval_is_identity():
    dropout = Dropout(p=0.5)
    dropout.eval()

    x = Tensor(np.random.randn(10, 5))
    out = dropout(x)

    np.testing.assert_allclose(out.data, x.data)


# ----------------------
# Sequential Tests
# ----------------------

def test_sequential_forward_chain():
    model = Sequential(
        Linear(4, 6),
        Linear(6, 2),
    )

    x = Tensor(np.random.randn(8, 4))
    out = model(x)

    assert out.shape == (8, 2)


def test_sequential_backward_pass():
    model = Sequential(
        Linear(3, 5),
        Linear(5, 1),
    )

    x = Tensor(np.random.randn(7, 3), requires_grad=True)
    out = model(x)
    loss = out.sum()
    loss.backward()

    assert x.grad is not None


# ----------------------
# Batch Normalization Tests
# ----------------------

def test_batchnorm_forward_shape():
    bn = BatchNormalization(num_features=4)
    x = Tensor(np.random.randn(10, 4))

    out = bn(x)

    assert out.shape == x.shape


def test_batchnorm_running_stats_update():
    bn = BatchNormalization(num_features=3)
    bn.train()

    x = Tensor(np.random.randn(20, 3))

    running_mean_before = bn.running_mean.data.copy()
    _ = bn(x)

    assert not np.allclose(running_mean_before, bn.running_mean.data)


def test_batchnorm_eval_uses_running_stats():
    bn = BatchNormalization(num_features=2)

    x = Tensor(np.random.randn(10, 2))
    _ = bn(x)  # update stats

    bn.eval()
    out1 = bn(x)
    out2 = bn(x)

    np.testing.assert_allclose(out1.data, out2.data)


# ----------------------
# Layer Normalization Tests
# ----------------------

def test_layernorm_forward_shape():
    ln = LayerNormalization(4)
    x = Tensor(np.random.randn(5, 4))

    out = ln(x)

    assert out.shape == x.shape


def test_layernorm_zero_mean_unit_var():
    ln = LayerNormalization(6)
    x = Tensor(np.random.randn(12, 6))

    out = ln(x)

    mean = out.data.mean(axis=-1)
    var = out.data.var(axis=-1)

    np.testing.assert_allclose(mean, np.zeros_like(mean), atol=1e-5)
    np.testing.assert_allclose(var, np.ones_like(var), atol=1e-4)


def test_layernorm_backward_pass():
    ln = LayerNormalization(3)
    x = Tensor(np.random.randn(4, 3), requires_grad=True)

    out = ln(x)
    loss = out.sum()
    loss.backward()

    assert x.grad is not None
    assert ln.weight.grad is not None
    assert ln.bias.grad is not None

test_linear_forward_shape()
test_linear_backward_pass()

